Script per la creazione del dataset P05 (Anagrafica CIG PNRR/PNC 2023)

In [1]:
import pandas as pd
import numpy as np 

Descrizione processo Borruso
- Scarica tutti i csv zippati con i delta aggiornamenti di CIG bandi gara (ANAC, 23, dicembre compreso)
- Decomprime e converte in gzip così sono accessibili in stream
- Tra tutti i file scaricati, filtra soltanto quelli con flag PNRR uguale a 1
- Per CIG duplicati, preso il più recente
- Merge di tutti i file in uno solo

Descrizione processo ricreato
- Scarica tutti i csv zippati con i CIG bandi gara mensili (ANAC, 2023, dicembre compreso)
- Unzip i csv, trasforma in pandas dataframe e concatena i dataset mensili in un unico dataset annuale
- Filtra solo le righe con FLAG_PNRR==1

In [2]:
import urllib.request
import zipfile

In [6]:
%pip install -U bs4

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [42]:
from bs4 import BeautifulSoup
import requests
import re
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

html_page = requests.get('https://dati.anticorruzione.it/opendata/dataset/cig-2023')
soup = BeautifulSoup(html_page.content, 'html.parser')

# find all a href links available on this page
file_urls = soup.findAll('a', class_="heading",title=re.compile("^cig_csv"))
links = [link.get('href') for link in file_urls]

In [43]:
links

['/opendata/dataset/cig-2023/resource/009c2db6-1a81-49ca-b79d-d0c1d54f4d9d',
 '/opendata/dataset/cig-2023/resource/368e6321-f985-4979-a7ce-2b14c46021af',
 '/opendata/dataset/cig-2023/resource/ef14b555-67fc-4c9b-8b66-949c0413aeee',
 '/opendata/dataset/cig-2023/resource/ab62f96d-eba2-4e28-9a3b-49fea6641aa0',
 '/opendata/dataset/cig-2023/resource/5491aec8-c7c4-4a36-a18a-423bcee3bd20',
 '/opendata/dataset/cig-2023/resource/eefc7fd8-ef47-4e65-8a93-289be78b38a3',
 '/opendata/dataset/cig-2023/resource/489dc99b-25ed-4efb-b7f8-3dc68f524cdd',
 '/opendata/dataset/cig-2023/resource/55bb5537-30c7-474f-a575-f51845d79f28',
 '/opendata/dataset/cig-2023/resource/bb9dc902-0112-4688-ab2e-c68bf1ff0c0d',
 '/opendata/dataset/cig-2023/resource/8ff4d215-8c35-4b2b-aaad-c6bb486a07b3',
 '/opendata/dataset/cig-2023/resource/504c5a2e-8f5a-4ad2-a621-0ed871bc9a96',
 '/opendata/dataset/cig-2023/resource/bf300c34-38fb-42c6-a792-9bb84fa7af0f']

In [44]:
d = []
for link in links:
    l = 'https://dati.anticorruzione.it'+link
    html_page = requests.get(l)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    file_urls = soup.findAll('a',class_="btn btn-primary resource-url-analytics resource-type-None")
    d.append(file_urls[0].get('href'))

In [47]:
d
#d[8:]

['https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_01.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_02.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_03.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_04.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_05.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_06.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_07.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_08.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023_09.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig-2023/filesystem/cig_csv_2023

In [69]:
# initiating a dataframe
full_df = pd.DataFrame()

for link in d:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[0]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df = pd.concat([full_df, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

cig_csv_2023_01.csv Completed
cig_csv_2023_02.csv Completed
cig_csv_2023_03.csv Completed
cig_csv_2023_04.csv Completed
cig_csv_2023_05.csv Completed
cig_csv_2023_06.csv Completed
cig_csv_2023_07.csv Completed
cig_csv_2023_08.csv Completed
cig_csv_2023_09.csv Completed
cig_csv_2023_10.csv Completed
cig_csv_2023_11.csv Completed
cig_csv_2023_12.csv Completed


In [70]:
# initiating a dataframe
full_df1 = pd.DataFrame()

for link in d[8:]:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[0]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df1 = pd.concat([full_df1, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

cig_csv_2023_09.csv Completed
cig_csv_2023_10.csv Completed
cig_csv_2023_11.csv Completed
cig_csv_2023_12.csv Completed


In [71]:
print(full_df['cig'].nunique())
full_df1['cig'].nunique()

583920


188933

In [72]:
p5_prova = full_df[full_df['FLAG_PNRR_PNC']=='1']
p5_prova1 = full_df1[full_df1['FLAG_PNRR_PNC']=='1']

In [73]:
print(p5_prova.shape)
print(p5_prova1.shape)
print(p5_prova['cig'].nunique())
print(p5_prova1['cig'].nunique())

(170481, 61)
(58600, 61)
170481
58600


Perché i CIG unici PNRR sono meno di prima? 

In [14]:
p5_prova.to_csv("p5_prova.csv", index = False, sep=';')

In [3]:
p5_prova = pd.read_csv("p5_prova.csv", sep=';')

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_26392\4076170598.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  p5_prova = pd.read_csv("p5_prova.csv", sep=';')


In [74]:
p5_prova1.nunique()

cig                         58600
cig_accordo_quadro            894
numero_gara                 56718
oggetto_gara                52687
importo_complessivo_gara    34245
                            ...  
CIG_COLLEGAMENTO              370
COD_ESITO                       6
ESITO                           6
DATA_COMUNICAZIONE_ESITO      124
FLAG_PNRR_PNC                   1
Length: 61, dtype: int64

In [3]:
pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_1268\3974010981.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")


In [21]:
pnew["data_pubblicazione"].unique()

array(['2022-11-24', '2023-03-17', '2023-01-02', ..., '2021-10-21',
       '2007-03-19', '2021-12-19'], dtype=object)

In [22]:
pnew['FLAG_PNRR_PNC'].unique()

array([1], dtype=int64)

In [29]:
import re

In [38]:
pnew['file'].unique()

array(['20231101-cig_csv.gz', '20231001-cig_csv.gz'], dtype=object)

In [91]:
print("Bandi pubblicati nel 2019:",pnew[pnew['data_pubblicazione'].str.match('2019')].shape[0])
print("Bandi pubblicati nel 2020:",pnew[pnew['data_pubblicazione'].str.match('2020')].shape[0])
print("Bandi pubblicati nel 2021:",pnew[pnew['data_pubblicazione'].str.match('2021')].shape[0])
print("Bandi pubblicati nel 2022:",pnew[pnew['data_pubblicazione'].str.match('2022')].shape[0])
print("Bandi pubblicati nel 2023:",pnew[pnew['data_pubblicazione'].str.match('2023')].shape[0])

Bandi pubblicati nel 2019: 35
Bandi pubblicati nel 2020: 154
Bandi pubblicati nel 2021: 576
Bandi pubblicati nel 2022: 38515
Bandi pubblicati nel 2023: 137974


In [92]:
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5_prova[p5_prova['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5_prova[p5_prova['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5_prova[p5_prova['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5_prova[p5_prova['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5_prova[p5_prova['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 0
Bandi pubblicati nel 2021: 0
Bandi pubblicati nel 2022: 0
Bandi pubblicati nel 2023: 170481


Dataset con CIG pubblicati nel 2022

In [79]:
from bs4 import BeautifulSoup
import requests
import re
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

html_page = requests.get('https://dati.anticorruzione.it/opendata/dataset/cig-2022')
soup = BeautifulSoup(html_page.content, 'html.parser')


# find all a href links available on this page
file_urls = soup.findAll('a', class_="heading",title=re.compile("^cig_csv"))
links = [link.get('href') for link in file_urls]

d = []
for link in links:
    l = 'https://dati.anticorruzione.it'+link
    html_page = requests.get(l)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    file_urls = soup.findAll('a',class_="btn btn-primary resource-url-analytics resource-type-None")
    d.append(file_urls[0].get('href'))

In [80]:
# initiating a dataframe
full_df2 = pd.DataFrame()

for link in d:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[0]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df2 = pd.concat([full_df2, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

cig_csv_2022_01.csv Completed
cig_csv_2022_02.csv Completed
cig_csv_2022_03.csv Completed
cig_csv_2022_04.csv Completed
cig_csv_2022_05.csv Completed
cig_csv_2022_06.csv Completed
cig_csv_2022_07.csv Completed
cig_csv_2022_08.csv Completed
cig_csv_2022_09.csv Completed
cig_csv_2022_10.csv Completed
cig_csv_2022_11.csv Completed
cig_csv_2022_12.csv Completed


In [81]:
p5_prova2 = full_df2[full_df2['FLAG_PNRR_PNC']=='1']

In [93]:
print(p5_prova2['cig'].nunique())
print(pnew['FLAG_PNRR_PNC'].unique())
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5_prova2[p5_prova2['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5_prova2[p5_prova2['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5_prova2[p5_prova2['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5_prova2[p5_prova2['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5_prova2[p5_prova2['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

38939
[1]
Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 0
Bandi pubblicati nel 2021: 0
Bandi pubblicati nel 2022: 38939
Bandi pubblicati nel 2023: 0


In [94]:
p5 = pd.concat([p5_prova, p5_prova2], ignore_index=True, sort=False)

In [100]:
print(p5.shape[0])
print(p5['cig'].nunique())

209420
209420


In [101]:
p5 = p5.drop_duplicates()

In [104]:
print(p5['cig'].nunique())
print(p5['FLAG_PNRR_PNC'].unique())
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5[p5['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5[p5['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5[p5['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5[p5['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5[p5['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

209420
['1']
Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 0
Bandi pubblicati nel 2021: 0
Bandi pubblicati nel 2022: 38939
Bandi pubblicati nel 2023: 170481


In [109]:
p5.to_csv("p5.csv", index = False, sep=';')

2021

In [105]:
html_page = requests.get('https://dati.anticorruzione.it/opendata/dataset/cig-2021')
soup = BeautifulSoup(html_page.content, 'html.parser')


# find all a href links available on this page
file_urls = soup.findAll('a', class_="heading",title=re.compile("^cig_csv"))
links = [link.get('href') for link in file_urls]

d = []
for link in links:
    l = 'https://dati.anticorruzione.it'+link
    html_page = requests.get(l)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    file_urls = soup.findAll('a',class_="btn btn-primary resource-url-analytics resource-type-None")
    d.append(file_urls[0].get('href'))

# initiating a dataframe
full_df3 = pd.DataFrame()

for link in d:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[0]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df3 = pd.concat([full_df3, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')
full_df3

cig_csv_2021_01.csv Completed
cig_csv_2021_02.csv Completed
cig_csv_2021_03.csv Completed
cig_csv_2021_04.csv Completed
cig_csv_2021_05.csv Completed
cig_csv_2021_06.csv Completed
cig_csv_2021_07.csv Completed
cig_csv_2021_08.csv Completed
cig_csv_2021_09.csv Completed
cig_csv_2021_10.csv Completed
cig_csv_2021_11.csv Completed
cig_csv_2021_12.csv Completed


,cig,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,TIPO_APPALTO_RISERVATO,CUI_PROGRAMMA,FLAG_PREV_RIPETIZIONI,COD_IPOTESI_COLLEGAMENTO,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC
0,8758994681,NaN,8156101,DISPOSITIVO PER TERMOREGOLAZIONE DEI CAMPIONI ...,72000.0,1,DISPOSITIVO PER TERMOREGOLAZIONE DEI CAMPIONI ...,72000.0,FORNITURE,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2023-07-07,NaN
1,85934291F3,NaN,8016852,Oggetto non pubblicabile a causa della potenzi...,5995380.52,2,Oggetto non pubblicabile a causa della potenzi...,5582255.8,SERVIZI,ATTIVO,...,NaN,S01199250158201900070,1,NaN,NaN,NaN,1.0,AGGIUDICATA,2021-10-29,NaN
2,9002772A78,NaN,8369725,LAVORI DI SOSTITUZIONE RETE ACQUEDOTTO VIA ROS...,146828.93,1,LAVORI DI SOSTITUZIONE RETE ACQUEDOTTO VIA ROS...,146828.93,LAVORI,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2021-12-03,NaN
3,87222937E1,NaN,8122762,SISMA 2016 2017: RIPARAZIONE CON MIGLIORAMENTO...,377202.0,1,SISMA 2016 2017: RIPARAZIONE CON MIGLIORAMENTO...,377202.0,LAVORI,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8737258564,8347965EA2,8136009,FORNITURA VACCINO ANTIMENIGOCOCCICO B MULTICOM...,479000.0,1,FORNITURA VACCINO ANTIMENIGOCOCCICO B MULTICOM...,479000.0,FORNITURE,ATTIVO,...,NaN,01684950593202100001,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2021-05-18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428751,9066194BF5,NaN,8422098,LAVORI IN SOMMA URGENZA PER DANNI ALLUVIONALI ...,89046.0,1,LAVORI IN SOMMA URGENZA PER DANNI ALLUVIONALI ...,89046.0,LAVORI,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2022-02-16,NaN
428752,9067614FC6,8976958417,8423461,ID13SER011 SERVIZIO DI TRASPORTO DELLE PERSONE...,200000.0,1,ID13SER011 SERVIZIO DI TRASPORTO DELLE PERSONE...,200000.0,SERVIZI,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2023-01-24,NaN
428753,9085582B71,NaN,8437926,CONTRATTO PONTE DELIBERA N. 1663 DEL 30/12/202...,45000.0,1,CONTRATTO PONTE DELIBERA N. 1663 DEL 30/12/202...,45000.0,FORNITURE,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428754,90633565F9,NaN,8419407,ACQUISTO CARBURANTE PER AUTOTRAZIONE TRAMITE MEPA,1320000.0,1,ACQUISTO CARBURANTE PER AUTOTRAZIONE TRAMITE MEPA,1320000.0,FORNITURE,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
p5_prova3 = full_df3[full_df3['FLAG_PNRR_PNC']=='1']

In [107]:
print(p5_prova3['cig'].nunique())
print(p5_prova3['FLAG_PNRR_PNC'].unique())
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5_prova3[p5_prova3['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5_prova3[p5_prova3['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5_prova3[p5_prova3['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5_prova3[p5_prova3['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5_prova3[p5_prova3['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

660
['1']
Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 0
Bandi pubblicati nel 2021: 660
Bandi pubblicati nel 2022: 0
Bandi pubblicati nel 2023: 0


2020

In [110]:
html_page = requests.get('https://dati.anticorruzione.it/opendata/dataset/cig-2020')
soup = BeautifulSoup(html_page.content, 'html.parser')


# find all a href links available on this page
file_urls = soup.findAll('a', class_="heading",title=re.compile("^cig_csv"))
links = [link.get('href') for link in file_urls]

d = []
for link in links:
    l = 'https://dati.anticorruzione.it'+link
    html_page = requests.get(l)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    file_urls = soup.findAll('a',class_="btn btn-primary resource-url-analytics resource-type-None")
    d.append(file_urls[0].get('href'))

# initiating a dataframe
full_df4 = pd.DataFrame()

for link in d:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[0]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df4 = pd.concat([full_df4, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

p5_prova4 = full_df4[full_df4['FLAG_PNRR_PNC']=='1']

print(p5_prova4['cig'].nunique())
print(p5_prova4['FLAG_PNRR_PNC'].unique())
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5_prova4[p5_prova4['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5_prova4[p5_prova4['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5_prova4[p5_prova4['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5_prova4[p5_prova4['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5_prova4[p5_prova4['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

cig_csv_2020_01.csv Completed
cig_csv_2020_02.csv Completed
cig_csv_2020_03.csv Completed
cig_csv_2020_04.csv Completed
cig_csv_2020_05.csv Completed
cig_csv_2020_06.csv Completed
cig_csv_2020_07.csv Completed
cig_csv_2020_08.csv Completed
cig_csv_2020_09.csv Completed
cig_csv_2020_10.csv Completed
cig_csv_2020_11.csv Completed
cig_csv_2020_12.csv Completed
184
['1']
Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 184
Bandi pubblicati nel 2021: 0
Bandi pubblicati nel 2022: 0
Bandi pubblicati nel 2023: 0


In [111]:
p5tot = pd.concat([p5, p5_prova3], ignore_index=True, sort=False)
p5tot = pd.concat([p5tot, p5_prova4], ignore_index=True, sort=False)

In [114]:
p5tot.drop_duplicates()
print(p5tot['cig'].nunique())
print(p5tot['FLAG_PNRR_PNC'].unique())
# quanti bandi pubblicati nel 2020, 2021, 2022, 2023
print("Bandi pubblicati nel 2019:",p5tot[p5tot['data_pubblicazione'].str.match('2019')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2020:",p5tot[p5tot['data_pubblicazione'].str.match('2020')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2021:",p5tot[p5tot['data_pubblicazione'].str.match('2021')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2022:",p5tot[p5tot['data_pubblicazione'].str.match('2022')]['data_pubblicazione'].shape[0])
print("Bandi pubblicati nel 2023:",p5tot[p5tot['data_pubblicazione'].str.match('2023')]['data_pubblicazione'].shape[0])

210264
['1']
Bandi pubblicati nel 2019: 0
Bandi pubblicati nel 2020: 184
Bandi pubblicati nel 2021: 660
Bandi pubblicati nel 2022: 38939
Bandi pubblicati nel 2023: 170481


In [115]:
p5tot.to_csv("p5tot.csv", index = False, sep=';')